# Import Library

In [32]:
import numpy as np
import pandas as pd  
import math
import time
import warnings
import polars as pl
import re
import json
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt
import seaborn as sns

import plotly as py
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [33]:
path = 'raw_appendicitis.csv'
# usecols=['visits','rowID','cid','visitDateTime','icd10_list','diagnoses']
raw_diag_df = pd.read_csv(path, lineterminator='\n', converters={'diagnoses':json.loads})

In [34]:
raw_diag_df

,rowID,cid,patientDOB,patientAge,patientSexName,vital_height,vital_bw,vital_bmi,admission,visitDateTime,diagnoses,icd10_list,diagType_1,vital_fbs,vital_cc,vital_pe,vital_hpi,vital_temperature,vital_bpd,vital_bps,vital_peHeart,alcohol,exercise,smoking,narcotic,drugs,HCT,WBC,RBC,PLT,Neutrophil,Lymphocytes,MCV,MCH,MCHC,BUN,Creatinine,Leukocytes,Bacteria,Urine Albumin,Urine Bilirubin,Urine Ketone,Urine RBC,Urine WBC,Urine Color,Urine pH,Urine Sugar\r
0,2946013,b229579fa0cfbf08dc9cc3b6348c9aa3f49ff05ef354a1...,1979-01-01 00:00:00 UTC,42,0,0.0,71.5,0.000,NaN,2021-01-14 08:59:46,"[{'icd10': 'K36', 'icd10Name': 'Other appendic...",['K36'],['K36'],0.0,มาตามนัด ปวดท้องบางครั้ง ทานได้ ถ่ายปกติ \r...,NaN,NaN,36.0,86.0,134.0,NaN,NaN,NaN,NaN,NaN,"[{""qty"":""20"",""drugNondugName"":""AMOXICILLIN+CLA...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\r
1,3501338,b2dc4043ef9c533471864ccc599d446a6e36ee39a190ba...,1978-05-06 00:00:00 UTC,42,0,0.0,67.9,0.000,NaN,2021-04-07 19:13:37,"[{'icd10': 'K352', 'icd10Name': 'Acute appendi...",['K352'],['K352'],0.0,มาตรวจซ้ำ กลับบ้านไป ปวดท้องข้างขวามาก อาเจี...,NaN,NaN,36.1,74.0,113.0,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\r
2,1642704,46b1e31e7244630d54b24ffc05fabce15dd584efb7176a...,1986-01-01 00:00:00 UTC,34,0,0.0,0.0,0.000,NaN,2020-06-26 06:24:06,"[{'icd10': 'K352', 'icd10Name': 'Acute appendi...",['K352'],['K352'],0.0,แจ้งว่า ปวดท้อง อาเจียน 5ครั้ง ไม่มีถ่ายเหลว เ...,NaN,ปวดท้องมาก ตีสาม นอนไมได้ ทุรนทุราย\r\nอาเจียน...,37.2,88.0,132.0,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\r
3,4742246,b8964d1a32cee6db0db63fc0f2e889e4af3a539b96522f...,2015-04-20 00:00:00 UTC,6,1,126.0,28.1,17.700,NaN,2021-09-26 16:27:07,"[{'icd10': 'K352', 'icd10Name': 'Acute appendi...",['K352'],['K352'],0.0,16.35 น. ผู้ป่วยเดินมาเองมารดานำส่งแจ้งว่าผู้ป...,NaN,"มาด้วยไข้ปวดท้อง 1 วัน , ปวดท้องขวาล่างใต้สะดื...",39.0,61.0,80.0,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\r
4,2820769,d69c8df8a8ceb249bb7b4af12b174c940b6a42cfc967a8...,1969-03-15 00:00:00 UTC,51,0,0.0,49.1,0.000,NaN,2020-12-25 08:12:19,"[{'icd10': 'K352', 'icd10Name': 'Acute appendi...",['K352'],['K352'],0.0,มาตรวจตามนัด แผลที่หน้าท้อง แผลแห้งดี ปวด ...,PE :แผลแห้ง กินได้ ไม่ปวดท้อง Explore with ap...,NaN,36.6,81.0,147.0,NaN,No,NaN,No,No,"[{""qty"":""20"",""drugNondugName"":""AMOXICILLIN+CLA...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\r
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1431,2146005,1d60209df9b0437f25613530f9414481912d14c3de4607...,1974-10-15 00:00:00 UTC,45,1,0.0,62.0,0.000,NaN,2020-09-26 18:24:01,"[{'icd10': 'K37', 'icd10Name': 'Unspecified ap...",['K37'],['K37'],0.0,1 วันก่อนมา รพ. ปวดท้อง,1 วัน ปวดท้องขวาล่าง ไม่มีปวดที่อื่นมาก่อน คล...,18.19 น. ผู้ป่วยเดินมา/ผู้ป่วยให้ประวัติว่า 1 ...,36.9,78.0,118.0,NaN,NaN,NaN,NaN,NaN,[],37.2,21.23,6.17,414.0,88.5,6.5,60.3,19.7,32.7,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,\r
1432,3873437,bafa3aa41b99714d9a92bf444a1771c3116540cab1d238...,1989-09-06 00:00:00 UTC,31,0,0.0,57.0,0.000,NaN,2021-06-09 18:08:02,"[{'icd10': 'K37', 'icd10Name': 'Unspecified ap...","['K37', 'Z532']",['K37'],0.0,18.16น.ผู้ป่วยรถนั่ง มากับญาติ / มาจาก ER ส่ง...,หญิงสิทธิ ชำระเงินเอง\r\nปวดท้องด้านขวาร้าวไปส...,NaN,36.5,50.0,112.0,NaN,No,NaN,No,NaN,[],38.0,7.55,4.55,239.0,48.9,45.7,83.6,28.1,33.6,8.6,0.91,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,\r
1433,1466704,0d62ccf972e7c7562ebedc8014f7c76c00cbe9908869be...,2000-08-07 00:00:00 UTC,19,0,156.0,55.0,22.600,NaN,2020-05-26 12:42:39,"[{'icd10': 'K37', 'icd10Name': 'Unspecified ap...",['K37'],['K37'],0.0,"6ชม.ก่อนมารพ.ปวดท้องทั่วๆ , 3ชม.ย้ายมาปวดขวา ,...",NaN,3 ชม ก่อนมามีอาการปวดท้อง มีคลื้นไส้อาเจียน

In [35]:
df = raw_diag_df.explode('diagnoses').reset_index(drop=True)
preprocess_df = pd.concat([df ,pd.json_normalize(df['diagnoses'])],axis=1).drop(['diagnoses'],axis=1).reset_index(drop=True)
preprocess_df['icd10'] = preprocess_df['icd10'].str.upper()
preprocess_df

,rowID,cid,patientDOB,patientAge,patientSexName,vital_height,vital_bw,vital_bmi,admission,visitDateTime,icd10_list,diagType_1,vital_fbs,vital_cc,vital_pe,vital_hpi,vital_temperature,vital_bpd,vital_bps,vital_peHeart,alcohol,exercise,smoking,narcotic,drugs,HCT,WBC,RBC,PLT,Neutrophil,Lymphocytes,MCV,MCH,MCHC,BUN,Creatinine,Leukocytes,Bacteria,Urine Albumin,Urine Bilirubin,Urine Ketone,Urine RBC,Urine WBC,Urine Color,Urine pH,Urine Sugar\r,icd10,icd10Name,diagType,diagTypeName
0,2946013,b229579fa0cfbf08dc9cc3b6348c9aa3f49ff05ef354a1...,1979-01-01 00:00:00 UTC,42,0,0.0,71.5,0.000,NaN,2021-01-14 08:59:46,['K36'],['K36'],0.0,มาตามนัด ปวดท้องบางครั้ง ทานได้ ถ่ายปกติ \r...,NaN,NaN,36.0,86.0,134.0,NaN,NaN,NaN,NaN,NaN,"[{""qty"":""20"",""drugNondugName"":""AMOXICILLIN+CLA...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\r,K36,Other appendicitis,1,Principal Diagnosis
1,3501338,b2dc4043ef9c533471864ccc599d446a6e36ee39a190ba...,1978-05-06 00:00:00 UTC,42,0,0.0,67.9,0.000,NaN,2021-04-07 19:13:37,['K352'],['K352'],0.0,มาตรวจซ้ำ กลับบ้านไป ปวดท้องข้างขวามาก อาเจี...,NaN,NaN,36.1,74.0,113.0,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\r,K352,Acute appendicitis with generalized peritonitis,1,Principal Diagnosis
2,1642704,46b1e31e7244630d54b24ffc05fabce15dd584efb7176a...,1986-01-01 00:00:00 UTC,34,0,0.0,0.0,0.000,NaN,2020-06-26 06:24:06,['K352'],['K352'],0.0,แจ้งว่า ปวดท้อง อาเจียน 5ครั้ง ไม่มีถ่ายเหลว เ...,NaN,ปวดท้องมาก ตีสาม นอนไมได้ ทุรนทุราย\r\nอาเจียน...,37.2,88.0,132.0,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\r,K352,Acute appendicitis with generalized peritonitis,1,Principal Diagnosis
3,4742246,b8964d1a32cee6db0db63fc0f2e889e4af3a539b96522f...,2015-04-20 00:00:00 UTC,6,1,126.0,28.1,17.700,NaN,2021-09-26 16:27:07,['K352'],['K352'],0.0,16.35 น. ผู้ป่วยเดินมาเองมารดานำส่งแจ้งว่าผู้ป...,NaN,"มาด้วยไข้ปวดท้อง 1 วัน , ปวดท้องขวาล่างใต้สะดื...",39.0,61.0,80.0,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\r,K352,Acute appendicitis with generalized peritonitis,1,Principal Diagnosis
4,2820769,d69c8df8a8ceb249bb7b4af12b174c940b6a42cfc967a8...,1969-03-15 00:00:00 UTC,51,0,0.0,49.1,0.000,NaN,2020-12-25 08:12:19,['K352'],['K352'],0.0,มาตรวจตามนัด แผลที่หน้าท้อง แผลแห้งดี ปวด ...,PE :แผลแห้ง กินได้ ไม่ปวดท้อง Explore with ap...,NaN,36.6,81.0,147.0,NaN,No,NaN,No,No,"[{""qty"":""20"",""drugNondugName"":""AMOXICILLIN+CLA...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\r,K352,Acute appendicitis with generalized peritonitis,1,Principal Diagnosis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1957,1466704,0d62ccf972e7c7562ebedc8014f7c76c00cbe9908869be...,2000-08-07 00:00:00 UTC,19,0,156.0,55.0,22.600,NaN,2020-05-26 12:42:39,['K37'],['K37'],0.0,"6ชม.ก่อนมารพ.ปวดท้องทั่วๆ , 3ชม.ย้ายมาปวดขวา ,...",NaN,3 ชม ก่อนมามีอาการปวดท้อง มีคลื้นไส้อาเจียน 2 ...,36.5,62.0,121.0,NaN,NaN,NaN,NaN,NaN,[],39.0,17.6,5.1,329.0,72.8,19.4,76.5,26.7,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,1.0,1.0,0.0,7.0,0.0\r,K37,Unspecified appendicitis,1,Principal Diagnosis
1958,3498585,e168b61a4a8f370c7880ec302153ad463196d5cef2f1b2...,1996-07-24 00:00:00 UTC,24,1,0.0,97.0,0.000,NaN,2021-04-07 22:39:35,['K353'],['K353'],0.0,ปวดท้องข้างขวา มีไข้ เวียนศีรษะ เป็นก่อนมารพ 6...,sat 97%\r\nAlvaradoscore 8 คะแนน DDx -acute a...,6 ชมก่อนมารพ ปวดท้องด้านขวาปวดจี๊ดๆ เวียนศีรษ...,39.3,83.0,137.0,N,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\r,K353,Acute appendicitis with localized peritonitis,1,Principal Diagnosis
1959,1845430,15485864aca2d753ef1e4f1db274ea670e9e317d8f9a63...,1999-09-30 00:00:00 UTC,20,1,171.0,60.0,20.519,NaN,2020-08-05 06:25:10,"['9918', '99

# Data Pre-processing 
ขั้นตอนเตรียมข้อมูลเพื่อไปนำไปทำในขั้นตอนอื่นๆ 

In [36]:
preprocess_df.shape

(1962, 50)

In [37]:
preprocess_df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1962 entries, 0 to 1961
Data columns (total 50 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   rowID              1962 non-null   int64  
 1   cid                1962 non-null   object 
 2   patientDOB         1962 non-null   object 
 3   patientAge         1962 non-null   int64  
 4   patientSexName     1962 non-null   int64  
 5   vital_height       1945 non-null   float64
 6   vital_bw           1945 non-null   float64
 7   vital_bmi          1936 non-null   float64
 8   admission          579 non-null    object 
 9   visitDateTime      1962 non-null   object 
 10  icd10_list         1962 non-null   object 
 11  diagType_1         1962 non-null   object 
 12  vital_fbs          1920 non-null   float64
 13  vital_cc           1939 non-null   object 
 14  vital_pe           1309 non-null   object 
 15  vital_hpi          438 non-null    object 
 16  vital_temperature  1938 

In [38]:
preprocess_df.describe().T

,count,mean,std,min,25%,50%,75%,max
rowID,1962.0,2.741247e+06,1.875550e+06,342.00,1.081783e+06,2402877.00,4.457323e+06,6240731.0
patientAge,1962.0,3.596789e+01,2.045112e+01,2.00,1.800000e+01,33.00,5.300000e+01,91.0
patientSexName,1962.0,5.377166e-01,4.987025e-01,0.00,0.000000e+00,1.00,1.000000e+00,1.0
vital_height,1945.0,1.158442e+02,7.120270e+01,0.00,0.000000e+00,156.00,1.650000e+02,205.0
vital_bw,1945.0,5.047692e+01,3.270791e+01,0.00,3.900000e+01,55.00,6.630000e+01,500.0
vital_bmi,1936.0,2.265131e+01,5.018541e+01,0.00,0.000000e+00,19.61,2.500000e+01,400.0
vital_fbs,1920.0,3.265625e-01,7.758679e+00,0.00,0.000000e+00,0.00,0.000000e+00,269.0
vital_temperature,1938.0,3.413333e+01,9.708459e+00,0.00,3.620000e+01,36.60,3.700000e+01,40.6
vital_bpd,1946.0,6.920606e+01,2.129350e+01,0.00,6.300000e+01,72.00,8.000000e+01,128.0
vital_bps,1946.0,1.143479e+02,3.500526e+01,0.00,1.040000e+02,118.00,1.337500e+02,204.0


In [39]:
# % of Missing values in each columns
(preprocess_df.isna().sum()/preprocess_df.shape[0]*100).round(2)

rowID                 0.00
cid                   0.00
patientDOB            0.00
patientAge            0.00
patientSexName        0.00
vital_height          0.87
vital_bw              0.87
vital_bmi             1.33
admission            70.49
visitDateTime         0.00
icd10_list            0.00
diagType_1            0.00
vital_fbs             2.14
vital_cc              1.17
vital_pe             33.28
vital_hpi            77.68
vital_temperature     1.22
vital_bpd             0.82
vital_bps             0.82
vital_peHeart        70.95
alcohol              59.07
exercise             99.69
smoking              57.39
narcotic             70.13
drugs                 0.00
HCT                  75.43
WBC                  76.50
RBC                  75.33
PLT                  75.43
Neutrophil           80.89
Lymphocytes          80.89
MCV                  75.43
MCH                  75.43
MCHC                 81.75
BUN                  85.63
Creatinine           85.42
Leukocytes           88.58
B

In [40]:
preprocess_df.shape

(1962, 50)

In [41]:
preprocess_df.duplicated(['cid']).sum()

848

In [42]:
preprocess_df = preprocess_df.drop_duplicates(['cid']).reset_index(drop=True)

In [43]:
preprocess_df.shape

(1114, 50)

In [44]:
preprocess_df.duplicated(['cid']).sum()

0

## Diagnoses        

### Icd10

In [45]:
preprocess_df['icd10'].value_counts()

K37        456
K353       327
K352       219
K36         36
9918        20
K351         7
9051         7
9082100      5
9929         4
R1049        3
Z000         3
K381         3
5794         2
K389         2
K388         2
R102         2
9357         2
R100         2
N832         1
Z480         1
K659         1
A419         1
R1048        1
R098         1
Z017         1
8926         1
K859         1
K382         1
8952         1
4823         1
Name: icd10, dtype: int64

### Icd10 list

In [46]:
preprocess_df['icd10_list'].value_counts()

['K37']                             393
['K353']                            243
['K352']                            191
['K36']                              29
['K353', '9918']                     25
                                   ... 
['K353', '9357', '9738', 'Z480']      1
['K353', '5794', '9396']              1
['K659', 'K352']                      1
['R102', '9918', 'K353']              1
['K37', 'Z532']                       1
Name: icd10_list, Length: 163, dtype: int64

### Diagnose type

In [47]:
preprocess_df['diagType'].value_counts()

1     1053
2       54
4        5
5        1
00       1
Name: diagType, dtype: int64

### Diagnose type name

In [48]:
preprocess_df['diagTypeName'].value_counts()

Principal Diagnosis                     1009
Comorbidity (โรคอื่นที่เป็นร่วมด้วย)      54
Principal Diagnosis (โรคหลัก)             44
Other (สาเหตุภายนอกอื่น ๆ)                 4
External Causes of Accident                1
Other (อื่น ๆ)                             1
                                           1
Name: diagTypeName, dtype: int64

In [49]:
preprocess_df[['icd10','icd10Name']].value_counts().head(50)

icd10    icd10Name                                                                           
K37      Unspecified appendicitis                                                                456
K353     Acute appendicitis with localized peritonitis                                           327
K352     Acute appendicitis with generalized peritonitis                                         219
K36      Other appendicitis                                                                       36
9918     Injection or infusion of electrolytes                                                    20
9051     Microscopic examination of blood (bacterial smear)                                        7
K351     Acute appendicitis with peritoneal abscess                                                6
9082100  เจาะเลือดจากหลอดเลือดดำ                                                                   5
9929     Injection or infusion of other therapeutic or prophylactic substance                     

In [50]:
icd10_df = preprocess_df.loc[preprocess_df['icd10'].str.contains(r'K3[5-8].*')][['cid','icd10','icd10Name','diagType']]

In [51]:
icd10_df

,cid,icd10,icd10Name,diagType
0,b229579fa0cfbf08dc9cc3b6348c9aa3f49ff05ef354a1...,K36,Other appendicitis,1
1,b2dc4043ef9c533471864ccc599d446a6e36ee39a190ba...,K352,Acute appendicitis with generalized peritonitis,1
2,46b1e31e7244630d54b24ffc05fabce15dd584efb7176a...,K352,Acute appendicitis with generalized peritonitis,1
3,b8964d1a32cee6db0db63fc0f2e889e4af3a539b96522f...,K352,Acute appendicitis with generalized peritonitis,1
4,d69c8df8a8ceb249bb7b4af12b174c940b6a42cfc967a8...,K352,Acute appendicitis with generalized peritonitis,1
...,...,...,...,...
1109,ecfdcd42f922be2039228e9fb0f57cccbce0f0c42c9199...,K37,Unspecified appendicitis,1
1110,b6c83a611fbe7e2b69c13b09108bfecbc9924700f99278...,K36,Other appendicitis,1
1111,1d60209df9b0437f25613530f9414481912d14c3de4607...,K37,Unspecified appendicitis,1
1112,bafa3aa41b99714d9a92bf444a1771c3116540cab1d238...,K37,Unspecified appendicitis,1


In [52]:
icd10_df[['icd10','icd10Name']].value_counts()

icd10  icd10Name                                            
K37    Unspecified appendicitis                                 456
K353   Acute appendicitis with localized peritonitis            327
K352   Acute appendicitis with generalized peritonitis          219
K36    Other appendicitis                                        36
K351   Acute appendicitis with peritoneal abscess                 6
K381   Appendicular concretions                                   3
K388   Other specified diseases of appendix                       2
K389   Disease of appendix, unspecified                           2
K351   "WHOยกเลิก"Acute appendicitis with peritoneal abscess      1
K382   Diverticulum of appendix                                   1
dtype: int64

## Vital   

In [53]:
vitals_df = preprocess_df[["vital_cc", "vital_pe", "vital_hpi"]].loc[preprocess_df["vital_cc"].notnull() | preprocess_df["vital_pe"].notnull() | raw_diag_df["vital_hpi"].notnull()]

In [54]:
vitals_df

,vital_cc,vital_pe,vital_hpi
0,มาตามนัด ปวดท้องบางครั้ง ทานได้ ถ่ายปกติ \r...,NaN,NaN
1,มาตรวจซ้ำ กลับบ้านไป ปวดท้องข้างขวามาก อาเจี...,NaN,NaN
2,แจ้งว่า ปวดท้อง อาเจียน 5ครั้ง ไม่มีถ่ายเหลว เ...,NaN,ปวดท้องมาก ตีสาม นอนไมได้ ทุรนทุราย\r\nอาเจียน...
3,16.35 น. ผู้ป่วยเดินมาเองมารดานำส่งแจ้งว่าผู้ป...,NaN,"มาด้วยไข้ปวดท้อง 1 วัน , ปวดท้องขวาล่างใต้สะดื..."
4,มาตรวจตามนัด แผลที่หน้าท้อง แผลแห้งดี ปวด ...,PE :แผลแห้ง กินได้ ไม่ปวดท้อง Explore with ap...,NaN
...,...,...,...
1109,CC. ปวดท้องน้อยด้านขวามีไข้ เมื่อ 14 ชม. PTA...,ปวดท้องขวาล่าง 14 ชม. \r\n1 วัน ก่อนมารพ. ...,NaN
1110,11.26น.เดินมา ให้ประวัติ/ปวดท้องด้านขวาตั้งแเต...,"หญิง 67 ปี U/D HT, CKD\r\n1 d PTA ปวดท้องขวาล่...",NaN
1111,1 วันก่อนมา รพ. ปวดท้อง,1 วัน ปวดท้องขวาล่าง ไม่มีปวดที่อื่นมาก่อน คล...,18.19 น. ผู้ป่วยเดินมา/ผู้ป่วยให้ประวัติว่า 1 ...
1112,18.16น.ผู้ป่วยรถนั่ง มากับญาติ / มาจาก ER ส่ง...,หญิงสิทธิ ชำระเงินเอง\r\nปวดท้องด้านขวาร้าวไปส...,NaN


### Clinical

In [63]:
def extract_keywords_condition(row, patterns):
    for col in ["vital_cc", "vital_pe", "vital_hpi"]:
        text = row[col]
        if text:
            text = str(text).lower()
            match = re.search("|".join(patterns), text)
            if match:
                return match.group()
            else:
                return None
            
    return None

In [55]:
def extract_binary_condition(row, keywords_0, keywords_1):
    for col in ["vital_cc", "vital_pe", "vital_hpi"]:
        text = row[col]
        if text:
            text = text.lower()
            match_0 = re.search("|".join(keywords_0), text)
            match_1 = re.search("|".join(keywords_1), text)
            if match_0:
                return "Yes"
            elif match_1:
                return "No"
    return None

In [59]:
def extract_3_condition(row, keywords_0, keywords_1,keywords_2):
    for col in ["vital_cc", "vital_pe", "vital_hpi"]:
        text = row[col]
        if text:
            text = str(text).lower()
            match_0 = re.search("|".join(keywords_0), text)
            match_1 = re.search("|".join(keywords_1), text)
            match_2 = re.search("|".join(keywords_2), text)
            if match_0:
                return "0"
            elif match_1:
                return "1"
            elif match_2:
                return "2"
    return None

#### 1. Peritonitis/abdominal guarding ตำแหน่งความปวดท้อง (การอักเสบของเยื่อบุช่องท้อง)  
Categorical: (no / localized / generalized)                                           

In [57]:
patterns = ['ปวด.*',
           'ท้อง.*',
           '.*ท้อง',
           'ปวด.*ท้อง.*',
           'ปวด.*ท้อง.*ข้าง',
           'ปวด.*ท้อง.*ด้าน',
           'peritonitis',
           'peritonitis.*',
           '.*peritonitis',
           '.*peritonitis',
           'abdominal',
           '.*abdominal',
           'abdominal.*',
           'abdominal guarding',
           '.*abdominal guarding',
           'abdominal guarding.*',
           ]
perit_abdominal_pattern = r'|'.join(patterns)
vitals_df.loc[vitals_df["vital_cc"].str.contains(perit_abdominal_pattern, na=False) | vitals_df["vital_pe"].str.contains(perit_abdominal_pattern, na=False) | vitals_df["vital_hpi"].str.contains(perit_abdominal_pattern, na=False)].sample(20)

,vital_cc,vital_pe,vital_hpi
373,12.06น. ผู้ป่วยเดินมากับญาติ ให้ประวัติ มีอาก...,4วันก่อนมารพ. ปวดท้องล่างขวา คลื่นไส้อาเจียน ถ...,NaN
1088,6 ซม. ปวดท้องด้านขวา คลื่นไส้อาเจียน 1 ครั้ง,6 ชั่วโมงก่อน ปวดท้องบริเวณลิ้นปี่ ย้ายมาขวาล่...,NaN
280,"ปวดท้องด้านขวา ถ่ายเป็นน้ำ 7 ครั้ง , คลื่น...",NaN,NaN
953,มาตรวจตามนัด แผลที่หน้าท้อง แผลแห้งดี ไม่ปว...,s/p appendectomy 7/63 \r\nwound: Ok\r\nd/c,NaN
47,F/U 1 wks Case S/P Appendectomy ยังไม่ตัดไหม ...,s/p appendectomy\r\nwound: ok\r\ntotal stitch ...,NaN
179,00.00 น. ไข้ หนาวสั่น,U/D ภูมิแพ้\r\n\r\ntoday\r\n23 hr PTA กลางคืนม...,21.55 น. เดินมาเอง /แฟนนำส่ง /ให้ประวัติว่า 00...
759,ญาตินำส่งแจ้งว่าผู้ป่วยปวดท้อง อาเจียน 1 ครั้ง...,NaN,12 ชม ปวดมวนท้องทั่วๆ ปวดตลอดเวลา ต่อมาย้ายมาป...
425,มาตามนัด ผ่าตัดไส้ติ่ง ทานได้ นัดตัดไหม ฟังผลp...,PATHO :Appendix : Acute appendicitis \r\nPE ...,NaN
4,มาตรวจตามนัด แผลที่หน้าท้อง แผลแห้งดี ปวด ...,PE :แผลแห้ง กินได้ ไม่ปวดท้อง Explore with ap...,NaN
1110,11.26น.เดินมา ให้ประวัติ/ปวดท้องด้านขวาตั้งแเต...,"หญิง 67 ปี U/D HT, CKD\r\n1 d PTA ปวดท้องขวาล่...",NaN


In [60]:
# Peritonitis/abdominal guarding
# Categorical: (no / localized / generalized)      
peritonitis_abdominal
# localized 
localized = ['ปวด.*ท้อง','ปวด.*ท้อง.*ข้าง','ปวด.*ท้อง.*ด้าน','ปวด.*ท้องน้อย','.*ท้องน้อย']
# no  
no_perit_abdominal =  ['ไม่.*ปวดท้อง']
# generalized
generalized = ['ปวด.*ท้องทั่ว']

preprocess_df["Peritonitis/abdominal guarding"] = preprocess_df.apply(lambda x: extract_3_condition(x, no_perit_abdominal,localized,generalized), axis=1)

In [62]:
preprocess_df["Peritonitis/abdominal guarding"]

0       1
1       1
2       1
3       1
4       0
       ..
1109    1
1110    0
1111    1
1112    1
1113    1
Name: Peritonitis/abdominal guarding, Length: 1114, dtype: object

#### 2. Migration of pain มีการเปลี่ยนตำแหน่งความปวด?
Binary: no / yes                                                            

#### 3. Tenderness in right lower quadrant (RLQ) ระดับความปวดหน้าท้อง ขวา ล่าง
Binary: no / yes                      

#### 4. Rebound tenderness ระดับความปวดของการกดหน้าท้อง
Binary: no / yes                                                            

#### 5. Cough tenderness อาการไอ? (การเคลื่อนไหวของเยื่อบุช่องท้อง)                                                     
Binary: no / yes 

#### 6. Psoas sign ปวดท้อง จากการโยกสะโพก? (การเคลื่อนไหวของเยื่อบุช่องท้อง)                                         
Binary: negative / positive

#### 7. Nausea/vomiting คลื่นไส้ / อาเจียน?
Binary: no / yes 

#### 8. Anorexia อากาเบื่ออาหาร?
Binary: no / yes 

#### 9. Dysuria เจ็บตอนปัสสาวะ?
Binary: no / yes 

#### 10. Stool การขับถ่าย (ปกติ, ท้องเสีย,อาการท้องผูก)
Categorical: (normal / diarrhea/ obstipation)   

### Appendicitis Scores
#### Alvarado Score (AS)
- [x]  Migration of pain                             (1)                         
- [ ]  Anorexia / ketones in urine                   (1)
- [ ]  Nausea / emesis                               (1)
- [ ]  Tenderness in right lower quadrant            (2)
- [ ]  Rebound pain                                  (1)
- [ ]  Elevation of temperature (≥ 37.3 °C)          (1)
- [ ]  Leukocytosis (> 10,000/µl)                    (2)
- [ ]  Neutrophilia (> 75%)                          (1)

- Total                                              (10)

## Data Portability 
Obj: extract ['diagnoses', 'drugs']